In [3]:
import pandas as pd
import numpy as np

In [23]:
user = pd.read_csv('./whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('./whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('./whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('./whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
user_place = pd.read_csv('./whyout_data/user_place.csv', index_col=0) # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('./whyout_data/user_product.csv', index_col=0) # shape(31176,5834) 유저 x 상품 아이템
user_video = pd.read_csv('./whyout_data/user_video.csv', index_col=0) # shape (31177, 3250) 유저 x 영상 아이템

In [20]:
# user_product = user_product.apply(pd.to_numeric, errors='coerce')

In [22]:
# user_product.to_csv('updated_user_product.csv', index=True, encoding='utf-8-sig') 

In [6]:
# # idx missing value 찾기
# df = video.sort_values('idx')
# # 연속된 숫자 생성 (전체 범위의 시작과 끝을 기반으로)
# full_range = pd.Series(range(df['idx'].min(), df['idx'].max() + 1))
# # 누락된 숫자 찾기
# missing_numbers = full_range[~full_range.isin(df['idx'])]
# print(len(missing_numbers))

In [7]:
# # NaN 값 찾기
# nan_indices = np.where(place['주변환경'].isna())[0]
# count = 0

# if len(nan_indices) > 0:
#     print(f'총 행 수: {len(place)}, NaN 행 수: {len(nan_indices)}')
#     print("해당 열에서 NaN 값이 발견된 위치는 다음과 같습니다:")
#     for idx in nan_indices:
#         print(f"인덱스: {idx}, 값: NaN")
# else:
#     print("해당 열에 NaN 값이 없습니다.")

In [8]:
# # 행, 열 이름 변경
# new_index = [str(i+1) for i in range(user_video.shape[0])]
# user_video.index = new_index

# new_columns = [str(i+1) for i in range(user_video.shape[1])]
# user_video.columns = new_columns

# # 변경된 데이터프레임 저장
# user_video.to_csv('updated_user_video.csv', index=True, encoding='utf-8-sig') 

In [24]:
# 유저가 장소에 대한 평가 유무
# 0의 개수
zero_count = (user_place == 0).sum(axis=1)
total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_place != 0).sum(axis=1)
total_non_zero_count = non_zero_count.sum()

# 장소 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

print(f"0의 개수: {zero_count}")
print(f"0의 총 개수: {total_zero_count}")
print("=====================================")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"장소에 대한 평가가 없는 유저 수: {count_zero}")
print(f"장소에 대한 평가가 있는 유저 수: {count_non_zero}")

0의 개수: 1        4697
2        4697
3        4696
4        4697
5        4681
         ... 
31172    4697
31173    4697
31174    4697
31175    4697
31176    4697
Length: 31176, dtype: int64
0의 총 개수: 145966114
0이 아닌 숫자의 개수: 1         0
2         0
3         1
4         0
5        16
         ..
31172     0
31173     0
31174     0
31175     0
31176     0
Length: 31176, dtype: int64
0이 아닌 숫자의 총 개수: 467558
장소에 대한 평가가 없는 유저 수: 8756
장소에 대한 평가가 있는 유저 수: 22420


In [27]:
# 유저가 상품에 대한 평가 유무
# 0의 개수
zero_count = (user_product == 0).sum(axis=1)
total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_product != 0).sum(axis=1)
total_non_zero_count = non_zero_count.sum()

# 상품 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

print(f"0의 개수: {zero_count}")
print(f"0의 총 개수: {total_zero_count}")
print("=====================================")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"상품에 대한 평가가 없는 유저 수: {count_zero}")
print(f"상품에 대한 평가가 있는 유저 수: {count_non_zero}")

0의 개수: 1        5834
2        5834
3        5834
4        5834
5        5833
         ... 
31172    5830
31173    5834
31174    5834
31175    5834
31176    5833
Length: 31176, dtype: int64
0의 총 개수: 181873300
0이 아닌 숫자의 개수: 1        0
2        0
3        0
4        0
5        1
        ..
31172    4
31173    0
31174    0
31175    0
31176    1
Length: 31176, dtype: int64
0이 아닌 숫자의 총 개수: 7484
상품에 대한 평가가 없는 유저 수: 28180
상품에 대한 평가가 있는 유저 수: 2996


In [26]:
# 유저가 영상에 대한 평가 유무
# 0의 개수
zero_count = (user_video == 0).sum(axis=1)
total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_video != 0).sum(axis=1)
total_non_zero_count = non_zero_count.sum()

# 영상 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

print(f"0의 개수: {zero_count}")
print(f"0의 총 개수: {total_zero_count}")
print("=====================================")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"영상에 대한 평가가 없는 유저 수: {count_zero}")
print(f"영상에 대한 평가가 있는 유저 수: {count_non_zero}")

0의 개수: 1        3250
2        3247
3        3246
4        3250
5        3227
         ... 
31173    3249
31174    3250
31175    3250
31176    3250
31177    3250
Length: 31177, dtype: int64
0의 총 개수: 101214098
0이 아닌 숫자의 개수: 1         0
2         3
3         4
4         0
5        23
         ..
31173     1
31174     0
31175     0
31176     0
31177     0
Length: 31177, dtype: int64
0이 아닌 숫자의 총 개수: 111152
영상에 대한 평가가 없는 유저 수: 20110
영상에 대한 평가가 있는 유저 수: 11067
